In [1]:
import cv2
import time

import numpy as np
import face_recognition
import imutils
import pickle

In [2]:
videofile = 'vp-rotated.mov'
path_prefix = 'C:/Users/jshru_000/bootcamp1/FINAL/faces/'
pickle_path = path_prefix+'facesmeta.pkl'

In [3]:
data = []
with open(pickle_path, 'rb') as f:
    try:
        while True:
            data.append(pickle.load(f))
    except EOFError:
        pass

In [4]:
encdb = []
names = []
fnames = list(data[0].keys())
for fname in fnames:
    encdb.append(data[0][fname]['encoding'])
    names.append(data[0][fname]['name'])

In [5]:
def resizeframe(image, width=500):
    w = float(width)
    r = w / image.shape[1]
    dim = (width, int(image.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    xr = dim[0]/float(image.shape[1])
    yr = dim[1]/float(image.shape[0])
    return resized

In [6]:
def draw_str(dst, x, y, s, right=0):
    fontFace = cv2.FONT_HERSHEY_PLAIN
    fontScale = 2.0
    thickness = 2
    txtsize = cv2.getTextSize(s, fontFace, fontScale, thickness)
    text_width = txtsize[0][0]
    if right == 0:
        cv2.putText(dst, s, (x+1, y+1), fontFace, fontScale, (0, 0, 0), thickness = thickness, lineType=cv2.LINE_AA)
        cv2.putText(dst, s, (x, y), fontFace, fontScale, (26,71, 255), lineType=cv2.LINE_AA)
    else:
        wi = right - x
        if text_width > wi:
            scalefactor = wi/text_width
            fontScale *= scalefactor
        cv2.putText(dst, s, (x+1, y+1), fontFace, fontScale, (0, 0, 0), thickness = thickness, lineType=cv2.LINE_AA)
        cv2.putText(dst, s, (x, y), fontFace, fontScale, (26, 71, 255), lineType=cv2.LINE_AA)

In [7]:
def get_faces(frame):
    newframe = frame.copy()
    model = 'hog' # 'cnn'
    metadata = {}
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])
    boxes = face_recognition.face_locations(rgb,model=model)
    if not boxes == []:
        encodings = face_recognition.face_encodings(rgb, boxes)
        assert len(encodings) == len(boxes)
        for i, (top, right, bottom, left) in enumerate(boxes):
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)
            cv2.rectangle(newframe, (left, top), (right, bottom), (0, 255, 0), 2)
            #thisFace = frame[top:bottom,left:right]
            name = search_facedb(encodings[i])
            draw_str(newframe,left,top,name, right=right)
    else:
        draw_str(newframe,20,20,'No face identified')
    return(newframe)

In [8]:
def search_facedb(enc):
    tolerance = 0.5
    matches = face_recognition.compare_faces(encdb,enc,tolerance=tolerance)
    if True in matches:
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        for i in matchedIdxs:
            name = names[i]
            counts[name] = counts.get(name, 0) + 1
        name = max(counts, key=counts.get)
    else:
        name='Hi'
    return(name)

In [9]:
face_cascade_path = "haarcascade_frontalface_default.xml"
smile_cascade_path = "haarcascade_smile.xml"

face_cascade = cv2.CascadeClassifier(face_cascade_path)
smile_cascade = cv2.CascadeClassifier(smile_cascade_path)

In [10]:
video_capture = cv2.VideoCapture(0)
video_capture.set(3, 640)
video_capture.set(4, 480)

True

In [11]:

while True:
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame=get_faces(frame)     #name

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.05,
        minNeighbors = 22,
        minSize = (55, 55),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        smile = smile_cascade.detectMultiScale(
            roi_gray,
            scaleFactor = 1.8,
            minNeighbors=30,
            minSize = (25, 25),
            flags = cv2.CASCADE_SCALE_IMAGE
        )
        
        for (x, y, w, h) in smile:
            cv2.rectangle(roi_color, (x, y-20), (x+w, y+h), (255, 0, 0), 1)
#             cv2.imwrite(str(time.time()) + ".png", frame)
            cv2.putText(frame, "smiling", (10,40), cv2.FONT_ITALIC,1.5, (255, 0, 0), thickness =1, lineType=cv2.LINE_AA)
            exit(0)
        
    cv2.imshow('Real-time Smile Detect', frame)
    
    
#     if ret :
# #         frame = resizeframe(frame)
#         frame1 = frame.copy()
# #         cv2.imshow('Frames- pause p; resume r; quit q',frame)
#         if cv2.waitKey(1) & 0xFF == ord('p'): #pause
#             while (True):
#                 if flag:
#                     newframe = get_faces(frame1)
#                     cv2.imshow('Frames- pause p; resume r; quit q',frame1)
#                     flag = False
#                 if cv2.waitKey(1) & 0xFF == ord('r'): #resume
#                     flag = True
#                     break
# #         if cv2.waitKey(10) & 0xFF == ord('q'): #quit
# #             break
    
    

    if cv2.waitKey(1) == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
